In [0]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
!pip install mne==0.20.0
# CONNECT TO GOOGLE DRIVE: Note you need to have EEG folder into MyDrive
from google.colab import drive
drive.mount('/content/gdrive')

     |████████████████████████████████| 6.6MB 3.5MB/s 
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import mne
import numpy as np
import scipy, time
import scipy.io
import scipy.stats
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import sys

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

import mne
from mne.datasets import sample
from mne.decoding import (SlidingEstimator, GeneralizingEstimator,
                          cross_val_multiscore, LinearModel, get_coef)


In [0]:
tmp_rootdir = '/content/gdrive/My Drive/EEG/'

#path to resampled data
resampled_dir = tmp_rootdir + 'resampled_data/'

#path to store classifcation scores
# classify_dir = tmp_rootdir + "MNE/classification_logi_3electro/"
# fig_dir = classify_dir + "Figures/"

In [0]:
Mastoids = ['M1','M2']
EOG_list = ['HEOG', 'VEOG']
n_eeg_channels = 32

subj_list = ['001']
n_subj = len(subj_list) 
eeg_chan = []
eeg_chan = eeg_chan + ['A' + str(i+1) for i in range(32)]
chnames = eeg_chan + ['EXG' + str(i+1) for i in range(5)]
blocks = ['can', 'rev']
stimuli = ['standard', 'deviant']

biosemi_layout = mne.channels.read_custom_montage(tmp_rootdir + 'biosemi_cap_32_M_2_EOG_3.locs')
biosemi_layout.ch_names = chnames
event_id = {'standard/can':65321,
            'deviant/can':65322,
            'standard/rev':65341,
            'deviant/rev':65342,
            'can/beer':65391,
            'can/pier':65392,
            'canTest1':65401,
            'canTest2':65402,
            'rev/beer':65491,
            'rev/pier':65492,
            'revTest1':65501,
            'revTest2':65502
            }

In [0]:
##read the events from the event files instead of the stim channels in the data.
n_subj = len(subj_list)
classify_scores = np.zeros((91, 9))
ave_score_by_subj = []

In [0]:
subj = subj_list[0]

resampled_fname = resampled_dir+ "%s_resampled_raw.fif" %(subj)
raw = mne.io.read_raw_fif(resampled_fname, preload = True)
events = mne.find_events(raw)
raw.set_eeg_reference('average', projection = True)
baseline = (-0.2, 0.0)
epochs = mne.Epochs(raw, events = events, event_id=event_id, 
                    tmin = -0.2, tmax = 0.5, baseline=baseline, preload= True)

Opening raw data file /content/gdrive/My Drive/EEG/resampled_data/001_resampled_raw.fif...
Isotrak not found
    Range : 0 ... 638591 =      0.000 ...  4988.992 secs
Ready.
Reading 0 ... 638591  =      0.000 ...  4988.992 secs...
Trigger channel has a non-zero initial value of 130816 (consider using initial_event=True to detect this event)
2912 events found
Event IDs: [65321 65322 65341 65342 65391 65392 65401 65402 65491 65492 65501 65502]
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
2912 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Loading data for 2912 events and 91 original time points ...
0 bad epochs dropped


In [0]:
#FZ = epochs.ch_names.index(epochs.ch_names[0:32])
beer = epochs['can/beer'].crop(-0.2, 0.5)
pier = epochs['can/pier'].crop(-0.2, 0.5)
beer_raw = beer.get_data()[:, np.arange(32), :]
pier_raw = pier.get_data()[:, np.arange(32), :]
X = np.concatenate((beer_raw, pier_raw))
Y = np.repeat([0,1], [np.shape(beer_raw)[0], np.shape(pier_raw)[0]], axis = 0)

In [0]:
print(X.shape)
print(Y.shape)

(780, 32, 91, 1)
(780,)


In [0]:
x# Normalize values to be between 0 and 1

In [0]:
# Convert 2D data to 3D for CNN model to run.
X=X.reshape(len(X),len(X[0]),len(X[0][0]),1)

In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 91, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2))

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 89, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 44, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 42, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 21, 64)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 19, 64)         36928     
_________________________________________________________________
flatten (Flatten)            (None, 4864)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X, Y, epochs=10)

Epoch 1/10
25/25 [==============================] - 3s 116ms/step - loss: 0.6937 - accuracy: 0.4974
Epoch 2/10
25/25 [==============================] - 3s 113ms/step - loss: 0.6932 - accuracy: 0.5000
Epoch 3/10
25/25 [==============================] - 3s 113ms/step - loss: 0.6932 - accuracy: 0.4718
Epoch 4/10
25/25 [==============================] - 3s 114ms/step - loss: 0.6932 - accuracy: 0.4949
Epoch 5/10
25/25 [==============================] - 3s 115ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 6/10
25/25 [==============================] - 3s 115ms/step - loss: 0.6933 - accuracy: 0.5000
Epoch 7/10
25/25 [==============================] - 3s 114ms/step - loss: 0.6934 - accuracy: 0.5000
Epoch 8/10
25/25 [==============================] - 3s 115ms/step - loss: 0.6932 - accuracy: 0.5000
Epoch 9/10
25/25 [==============================] - 3s 115ms/step - loss: 0.6932 - accuracy: 0.5000
Epoch 10/10
25/25 [==============================] - 3s 114ms/step - loss: 0.6932 - accuracy: 0.5000

[Deep learning with cnn for EEG](https://https://arxiv.org/pdf/1703.05051.pdf)

In [0]:
# shallow CNN
conv_filters = (1, 15)
strides = (1, 3)
pool_size = (1, 35)
dropoutRate = 0.1

model = models.Sequential()
model.add(layers.Conv2D(40, conv_filters, input_shape=(1, 32, 91), activation='elu'))
model.add(layers.Conv2D(40, (1, 7), use_bias=False, activation='elu'))
model.add(layers.BatchNormalization(axis=1, epsilon=1e-05, momentum=0.1))
model.add(layers.MaxPooling2D((1, 2), strides=strides))
model.add(layers.Dropout(dropoutRate))
model.add(layers.Flatten())
model.add(layers.Dense(2))
model.add(layers.Activation('softmax'))

In [44]:
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_38 (Conv2D)           (None, 1, 18, 40)         54640     
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 1, 12, 40)         11200     
_________________________________________________________________
batch_normalization_21 (Batc (None, 1, 12, 40)         4         
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 1, 4, 40)          0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 1, 4, 40)          0         
_________________________________________________________________
flatten_21 (Flatten)         (None, 160)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 2)               

In [45]:
X = X.reshape(780,1,32,91)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X, Y, epochs=50)

Epoch 1/50
25/25 [==============================] - 0s 12ms/step - loss: 0.7491 - accuracy: 0.5256
Epoch 2/50
25/25 [==============================] - 0s 12ms/step - loss: 0.7190 - accuracy: 0.5077
Epoch 3/50
25/25 [==============================] - 0s 12ms/step - loss: 0.7207 - accuracy: 0.5064
Epoch 4/50
25/25 [==============================] - 0s 11ms/step - loss: 0.6914 - accuracy: 0.5372
Epoch 5/50
25/25 [==============================] - 0s 12ms/step - loss: 0.7015 - accuracy: 0.5090
Epoch 6/50
25/25 [==============================] - 0s 12ms/step - loss: 0.6978 - accuracy: 0.5231
Epoch 7/50
25/25 [==============================] - 0s 12ms/step - loss: 0.6378 - accuracy: 0.6462
Epoch 8/50
25/25 [==============================] - 0s 11ms/step - loss: 0.5927 - accuracy: 0.7000
Epoch 9/50
25/25 [==============================] - 0s 12ms/step - loss: 0.5575 - accuracy: 0.7436
Epoch 10/50
25/25 [==============================] - 0s 12ms/step - loss: 0.5463 - accuracy: 0.7615
Epoch 11/

In [51]:
Y_pred = np.zeros(len(Y))
i = 0
for y_hat,_ in model.predict(X):
    if y_hat >= 0.5:
        Y_pred[i] = 0
    else:
        Y_pred[i] = 1
    i += 1

from sklearn.metrics import classification_report
print(classification_report(Y, Y_pred, target_names=['beer', 'pier']))

              precision    recall  f1-score   support

        beer       0.90      0.84      0.87       390
        pier       0.85      0.91      0.87       390

    accuracy                           0.87       780
   macro avg       0.87      0.87      0.87       780
weighted avg       0.87      0.87      0.87       780

